In [ ]:
using LinearAlgebra
using Plots
using FFTW; # <-- This line is added to fix the error

# 1. Define model parameters
N = 500          # Number of atoms
m = 1.0          # Mass of atoms (arbitrary units)
C = 1.0          # Spring constant (arbitrary units)
a = 1.0          # Interatomic distance (arbitrary units)

# 2. Construct the dynamical matrix
# For periodic boundary conditions, the dynamical matrix is a circulant
# tridiagonal matrix.
# D_nn = 2C/m
# D_{n, n-1} = D_{n, n+1} = -C/m
# D_{1,N} = D_{N,1} = -C/m (due to periodicity)

D = zeros(Float64, N, N)

for i in 1:N
    D[i, i] = 2 * C / m
    if i > 1
        D[i, i-1] = -C / m
    end
    if i < N
        D[i, i+1] = -C / m
    end
end

# Apply periodic boundary conditions
D[1, N] = -C / m
D[N, 1] = -C / m

# 3. Solve the eigenvalue problem
# The eigvals function returns the eigenvalues of the matrix, which are ω²
#eigenvalues = eigvals(D)
eigenvalues, eigenvectors = eigen(Hermitian(D))

# Calculate the angular frequencies ω by taking the positive square root.
# A small tolerance (max(0.0, ...)) is used to prevent domain errors
# from tiny negative numbers that might arise from numerical precision issues.
#omegas = sqrt.(max.(0.0, eigenvalues))

# 4. Generate the wave vectors (k)
# For a chain of N atoms with periodic boundary conditions, the allowed
# wave vectors are quantized. We use fftshift to arrange them in ascending order.
k_values = 2 * π .*abs.(fftfreq( N , a))
k_values = range(0,N)


# 5. Plot the phonon dispersion curve
# Sort the calculated frequencies to match the ordered k-values for a clean plot.
#sorted_omegas = sort(omegas)

# Generate the analytical dispersion curve for comparison
k_analytical = range(-π/a, π/a, length=200)
omega_analytical = @. sqrt(4*C/m) * abs(sin(k_analytical * a / 2))

# Create the plot
plot(
    k_values,
    eigenvalues,
    seriestype = :scatter,
    label = "Simulation",
    xlabel = "Wave Vector (k)",
    ylabel = "Angular Frequency (ω)",
    title = "Phonon Dispersion in a 1D Atomic Chain",
    legend = :bottomright,
    markersize = 3,
    markerstrokewidth = 0
)

# Overlay the theoretical curve
#plot!(
#    k_analytical,
#    omega_analytical,
#    linewidth = 2,
#    label = "Analytical Theory",
#    linestyle = :dash
#)

# Mark the boundaries of the first Brillouin zone
#vline!([-π/a, π/a], label="Brillouin Zone Edge", color=:gray, linestyle=:dot)

# To display the plot in a REPL or notebook
current() 

# Or save it to a file
#display()
#savefig("phonon_dispersion_1D_julia.png")

In [ ]:
eigenvalues

In [ ]:
plot(
    k_values,
    eigenvectors[500,:],
    seriestype = :scatter,
    label = "Simulation",
    xlabel = "Wave Vector (k)",
    ylabel = "Angular Frequency (ω)",
    title = "Phonon Dispersion in a 1D Atomic Chain",
    legend = :bottomright,
    markersize = 3,
    markerstrokewidth = 0
)
current()